https://colab.research.google.com/drive/1P9iQHtUH5KUbTVtp8B4-AopZzEEPE0lw?usp=sharing#scrollTo=4YS7XI2bZTyz

https://github.com/aub-mind/arabert

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip -q install optuna==2.3.0
!pip -q install transformers==4.2.1
!pip -q install farasapy
!pip -q install pyarabic
!git clone https://github.com/aub-mind/arabert

     |████████████████████████████████| 258 kB 8.3 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 80 kB 11.3 MB/s 
     |████████████████████████████████| 164 kB 72.2 MB/s 
     |████████████████████████████████| 75 kB 6.3 MB/s 
     |████████████████████████████████| 141 kB 75.7 MB/s 
     |████████████████████████████████| 111 kB 80.1 MB/s 
     |████████████████████████████████| 49 kB 7.8 MB/s 
     |████████████████████████████████| 1.8 MB 9.1 MB/s 
     |████████████████████████████████| 2.9 MB 59.6 MB/s 
     |████████████████████████████████| 895 kB 62.4 MB/s 
     |████████████████████████████████| 126 kB 8.3 MB/s 
Cloning into 'arabert'...
remote: Enumerating objects: 544, done.
remote: Counting objects: 100% (330/330), done.
remote: Compressing objects: 100% (237/237), done.
remote: Total 544 (delta 175), reused 237 (delta 87), pack-reused 214
Receiving

In [ ]:
import pandas as pd
import numpy as np

from tqdm import tqdm_notebook as tqdm
from sklearn.model_selection import train_test_split

In [ ]:
hard_map = { 0 : 'fake', 1 : 'real', }

df_train = pd.read_csv("drive/My Drive/Arabic_FakeNews/Model/Balance/DataDescr/train_news.csv", header=0,encoding='utf-8')
df_train.label= df_train.label.apply(lambda x: hard_map[x])

df_val = pd.read_csv("drive/My Drive/Arabic_FakeNews/Model/Balance/DataDescr/val_news.csv", header=0,encoding='utf-8')
df_val.label= df_val.label.apply(lambda x: hard_map[x])

label_list_HARD = ['fake', 'real']


# Trainer

In [ ]:
from arabert.preprocess import ArabertPreprocessor
import numpy as np
from sklearn.metrics import classification_report, accuracy_score, f1_score, confusion_matrix, precision_score , recall_score

from transformers import AutoConfig, AutoModelForSequenceClassification, AutoTokenizer, BertTokenizer
from transformers.data.processors import SingleSentenceClassificationProcessor
from transformers import Trainer , TrainingArguments
from transformers.trainer_utils import EvaluationStrategy
from transformers.data.processors.utils import InputFeatures


In [ ]:
#https://huggingface.co/aubmindlab/araelectra-base-generator
dataset_name = 'NEWS'
model_name = 'aubmindlab/araelectra-base-discriminator'
task_name = 'classification'
max_len = 414

In [ ]:
arabert_prep = ArabertPreprocessor(model_name.split("/")[-1])

df_train.text = df_train.text.apply(lambda x:   arabert_prep.preprocess(x))
df_val.text = df_val.text.apply(lambda x:   arabert_prep.preprocess(x))  

In [ ]:
df_val.text

0       أطاحت الروسية داريا كاساتكينا بالفرنسية أليز ك...
1       قالت مصادر أمنية بشمال سيناء إن شرطيا مصريا قت...
2       عاجل قبل قليل ترامب يحس بالإغماء والشك بإصابته...
3       انا _ لله _ وإنا _ إليه _ راجعون توفيت صباح ال...
4       الفيتنامي فام دينه نوين ، وهو تاجر من مدينة هو...
                              ...                        
1277    الأوبئة تنتشر كل مئة عام ، في السنة ذات الرقم ...
1278    وزير الأوقاف : ندرس تقليل زمن الخطبة لصلاة الج...
1279    قالت صحيفة لوفيغارو إن وزارة الداخلية الفرنسية...
1280    بعد ما فاجئت العالم بإسلامها ، مريم بيترونين ت...
1281    نبه أطباء وأكاديميون بريطانيون إلى أن المشروبا...
Name: text, Length: 1282, dtype: object

In [ ]:
class BERTDataset():
    def __init__(self, text, target, model_name, max_len, label_map):
      super(BERTDataset).__init__()
      self.text = text
      self.target = target
      self.tokenizer_name = model_name
      self.tokenizer = AutoTokenizer.from_pretrained(model_name)
      self.max_len = max_len
      self.label_map = label_map
      

    def __len__(self):
      return len(self.text)

    def __getitem__(self,item):
      text = str(self.text[item])
      text = " ".join(text.split())


        
      input_ids = self.tokenizer.encode(
          text,
          add_special_tokens=True,
          max_length=self.max_len,
          truncation='longest_first'
      )     
    
      attention_mask = [1] * len(input_ids)

      # Zero-pad up to the sequence length.
      padding_length = self.max_len - len(input_ids)
      input_ids = input_ids + ([self.tokenizer.pad_token_id] * padding_length)
      attention_mask = attention_mask + ([0] * padding_length)    
      
      return InputFeatures(input_ids=input_ids, attention_mask=attention_mask, label=self.label_map[self.target[item]])

In [ ]:
label_map = { v:index for index, v in enumerate(label_list_HARD) }
print(label_map)
train_dataset = BERTDataset(df_train.text.to_list(),df_train.label.to_list(),model_name, max_len,label_map)
val_dataset  = BERTDataset(df_val.text.to_list()   ,df_val.label.to_list()  ,model_name,  max_len, label_map)

{'fake': 0, 'real': 1}


In [ ]:
def model_init():
    return AutoModelForSequenceClassification.from_pretrained(model_name, return_dict=True, num_labels=len(label_map))

In [ ]:
def compute_metrics(p): #p should be of type EvalPrediction
  preds = np.argmax(p.predictions, axis=1)
  assert len(preds) == len(p.label_ids)
  print(classification_report(p.label_ids,preds))
  print(confusion_matrix(p.label_ids,preds))

  # macro_f1_pos_neg = f1_score(p.label_ids,preds,average='macro',labels=[0,1])
  macro_f1 = f1_score(p.label_ids,preds,average='macro')
  macro_precision = precision_score(p.label_ids,preds,average='macro')
  macro_recall = recall_score(p.label_ids,preds,average='macro')
  acc = accuracy_score(p.label_ids,preds)
  # c_repoert= classification_report(p.label_ids,preds))

  return {
      'macro_f1' : macro_f1,
      # 'macro_f1_pos_neg' : macro_f1_pos_neg,  
      'macro_precision': macro_precision,
      'macro_recall': macro_recall,
      'accuracy': acc,
      # 'classification_report' :c_repoert
  }

# Regular Training

In [ ]:
training_args = TrainingArguments("./train")
training_args.evaluate_during_training = True
training_args.adam_epsilon = 1e-8
training_args.learning_rate = 5e-5
training_args.fp16 = True
# training_args.per_device_train_batch_size =16
# training_args.per_device_eval_batch_size = 16

training_args.per_device_train_batch_size = 16  
training_args.per_device_eval_batch_size = 16


training_args.gradient_accumulation_steps = 2
training_args.num_train_epochs= 8


steps_per_epoch = (len(df_train)// (training_args.per_device_train_batch_size * training_args.gradient_accumulation_steps))
total_steps = steps_per_epoch * training_args.num_train_epochs
print(steps_per_epoch)
print(total_steps)
#Warmup_ratio
warmup_ratio = 0.1
training_args.warmup_steps = total_steps*warmup_ratio # or you can set the warmup steps directly 

# training_args.load_best_model_at_end=True


training_args.evaluation_strategy = EvaluationStrategy.EPOCH
# training_args.logging_steps = 200
training_args.save_steps = 100000 #don't want to save any model, there is probably a better way to do this :)
training_args.seed = 42
training_args.disable_tqdm = False
training_args.lr_scheduler_type = 'cosine'

160
1280


In [ ]:
import torch
torch.cuda.empty_cache()

In [ ]:
trainer = Trainer(
    model = model_init(),
    args = training_args,
    train_dataset = train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics,
) 

Some weights of the model checkpoint at aubmindlab/araelectra-base-discriminator were not used when initializing ElectraForSequenceClassification: ['discriminator_predictions.dense.weight', 'discriminator_predictions.dense.bias', 'discriminator_predictions.dense_prediction.weight', 'discriminator_predictions.dense_prediction.bias']
- This IS expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at aubmindlab/araelectra-base-discriminator and are newly initialized: 

In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss,Macro F1,Macro Precision,Macro Recall,Accuracy,Runtime,Samples Per Second
0,No log,0.199891,0.921290,0.957954,0.895829,0.942278,20.464800,62.644000
1,No log,0.096559,0.969692,0.977255,0.962745,0.976599,20.441800,62.715000
2,No log,0.137317,0.966367,0.978669,0.955596,0.974259,20.414100,62.800000
3,0.130900,0.098765,0.975191,0.973893,0.976510,0.980499,20.485500,62.581000
4,0.130900,0.144541,0.971713,0.979307,0.964735,0.978159,20.556700,62.364000
5,0.130900,0.153246,0.969634,0.978230,0.961820,0.976599,20.379200,62.907000
6,0.007100,0.144068,0.972801,0.978890,0.967118,0.978939,20.408900,62.816000
7,0.007100,0.144657,0.972801,0.978890,0.967118,0.978939,20.326700,63.070000


/usr/local/lib/python3.7/dist-packages/transformers/trainer.py:911: FutureWarning: Non-finite norm encountered in torch.nn.utils.clip_grad_norm_; continuing anyway. Note that the default behavior will change in a future release to error out if a non-finite total norm is encountered. At that point, setting error_if_nonfinite=false will be required to retain the old behavior.
  self.args.max_grad_norm,


              precision    recall  f1-score   support

           0       0.99      0.80      0.88       343
           1       0.93      1.00      0.96       939

    accuracy                           0.94      1282
   macro avg       0.96      0.90      0.92      1282
weighted avg       0.95      0.94      0.94      1282

[[273  70]
 [  4 935]]
              precision    recall  f1-score   support

           0       0.98      0.93      0.96       343
           1       0.98      0.99      0.98       939

    accuracy                           0.98      1282
   macro avg       0.98      0.96      0.97      1282
weighted avg       0.98      0.98      0.98      1282

[[320  23]
 [  7 932]]
              precision    recall  f1-score   support

           0       0.99      0.92      0.95       343
           1       0.97      1.00      0.98       939

    accuracy                           0.97      1282
   macro avg       0.98      0.96      0.97      1282
weighted avg       0.97     

TrainOutput(global_step=1280, training_loss=0.05394646889180876, metrics={'train_runtime': 1792.181, 'train_samples_per_second': 0.714, 'total_flos': 13774160747798208, 'epoch': 8.0})

In [ ]:
trainer.evaluate(val_dataset)

              precision    recall  f1-score   support

           0       0.98      0.94      0.96       343
           1       0.98      0.99      0.99       939

    accuracy                           0.98      1282
   macro avg       0.98      0.97      0.97      1282
weighted avg       0.98      0.98      0.98      1282

[[323  20]
 [  7 932]]


{'epoch': 8.0,
 'eval_accuracy': 0.9789391575663027,
 'eval_loss': 0.14465686678886414,
 'eval_macro_f1': 0.9728014847840282,
 'eval_macro_precision': 0.978889737713267,
 'eval_macro_recall': 0.9671181115074967,
 'eval_runtime': 20.3564,
 'eval_samples_per_second': 62.978}

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_name)
trainer._save("/content/drive/MyDrive/Arabic_FakeNews/Model/Balance/SavedModels/AraElectra")
tokenizer.save_pretrained("/content/drive/MyDrive/Arabic_FakeNews/Model/Balance/SavedModels/AraElectra")

('/content/drive/MyDrive/Arabic_FakeNews/Model/Balance/SavedModels/AraElectra/tokenizer_config.json',
 '/content/drive/MyDrive/Arabic_FakeNews/Model/Balance/SavedModels/AraElectra/special_tokens_map.json',
 '/content/drive/MyDrive/Arabic_FakeNews/Model/Balance/SavedModels/AraElectra/vocab.txt',
 '/content/drive/MyDrive/Arabic_FakeNews/Model/Balance/SavedModels/AraElectra/added_tokens.json')

# Test

In [ ]:
df_test = pd.read_csv("drive/My Drive/Arabic_FakeNews/Model/Balance/DataDescr/test_news.csv", header=0,encoding='utf-8')

In [ ]:
df_test.text = df_test.text.apply(lambda x:   arabert_prep.preprocess(x))

In [ ]:
df_test.label= df_test.label.apply(lambda x: hard_map[x])
test_dataset = BERTDataset(df_test.text.to_list(),df_test.label.to_list(),model_name, max_len,label_map)


In [ ]:
trainer.evaluate(test_dataset)

              precision    recall  f1-score   support

           0       0.98      0.92      0.95       400
           1       0.97      0.99      0.98      1203

    accuracy                           0.97      1603
   macro avg       0.98      0.96      0.97      1603
weighted avg       0.97      0.97      0.97      1603

[[ 368   32]
 [   9 1194]]


{'epoch': 8.0,
 'eval_accuracy': 0.974422956955708,
 'eval_loss': 0.16461922228336334,
 'eval_macro_f1': 0.9651767865024349,
 'eval_macro_precision': 0.975013089514974,
 'eval_macro_recall': 0.9562593516209477,
 'eval_runtime': 25.8824,
 'eval_samples_per_second': 61.934}

# Predict

In [ ]:
class Dataset():
    def __init__(self, text=None, target= None, model_name = None, max_len= None, label_map= None):
      super(BERTDataset).__init__()
      self.text = text
      self.target = target
      self.tokenizer_name = model_name
      self.tokenizer = AutoTokenizer.from_pretrained(model_name)
      self.max_len = max_len
      self.label_map = label_map
      

    def __len__(self):
      return len(self.text)

    def __getitem__(self,item):
      text = str(self.text[item])
      text = " ".join(text.split())


        
      input_ids = self.tokenizer.encode(
          text,
          add_special_tokens=True,
          max_length=self.max_len,
          truncation='longest_first'
      )     
    
      attention_mask = [1] * len(input_ids)

      # Zero-pad up to the sequence length.
      padding_length = self.max_len - len(input_ids)
      input_ids = input_ids + ([self.tokenizer.pad_token_id] * padding_length)
      attention_mask = attention_mask + ([0] * padding_length)    
      
      return InputFeatures(input_ids=input_ids, attention_mask=attention_mask)

In [ ]:

title_example = pd.read_csv("drive/My Drive/Arabic_FakeNews/Model/Balance/Data/title_example.csv")
title_example.shape

count=0
for title in title_example.text:
   test_dataset = Dataset([title],None,model_name, max_len,label_map)
   y_label,_,_ = trainer.predict(test_dataset)
   preds = np.argmax(y_label, axis=1)   
   print(preds)
   if preds==1 :
    count=count+1

    
print("Real : "+str(count))

[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[1]
[0]
[0]
[0]
[0]
[1]
[0]
[0]
[0]
[0]
[0]
[0]
Real : 2


In [ ]:
text_example = pd.read_csv("drive/My Drive/Arabic_FakeNews/Model/Balance/Data/text_example.csv")
text_example.shape
count=0

for text in text_example.text:
   text_dataset = Dataset([text],None,model_name, max_len,label_map)
   y_label,_,_ = trainer.predict(text_dataset)
   preds = np.argmax(y_label, axis=1)   
   print(preds)
   if preds==1 :
    count=count+1
    
print("Real : "+str(count))

[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
Real : 0


In [ ]:
#  ----- 3. Predict -----#
# # Load test data
# test_data = pd.("drive/My Drive/Arabic_FakeNews/Model/Data/test_news.csv", header=0,encoding='utf-8')
# X_test = list(test_data["text"])
# X_test_tokenized = tokenizer(X_test, padding=True, truncation=True, max_length=512)

# # Create torch dataset
# test_dataset = Dataset(X_test_tokenized)

# # Load trained model
# model_path = "output/checkpoint-50000"
# model = BertForSequenceClassification.from_pretrained(model_path, num_labels=2)

# # Define test trainer
# test_trainer = Trainer(model)

# # Make prediction
# raw_pred, _, _ = test_trainer.predict(test_dataset)

# # Preprocess raw predictions
# y_pred = np.argmax(raw_pred, axis=1)